# Week 1 Homework
## Example 20-5 MWR
### Theoretical stoichiometric calculation for Fe2+ and Mn2+ removal using KMnO4

> *This example is taken from Chapter 20 of MWH's Water Treatment: Principles and Design (3rd ed.; DOI: 10.1002/9781118131473)*

A groundwater containing 5 g/m3 Fe2+ and 2 g/m3 Mn2+ is processed at a flow rate of 100,000 m3/day.

Potassium permanganate (KMnO4) is used to oxidize Fe2+ and Mn2+.

Calculate the quantity (kg/d *and* lb/hr) of potassium permanganate required, alkalinity consumed as CaCO3, and quantity of sludge produced in total and for each ion (kg/d).

Use the oxidation reactions for iron and manganese using KMnO4 as shown in Tables 20-8 and 20-10, respectively.

<img src="MWH_20-5-A.png">
<img src="MWH_20-5-B.png">

## Problem statement

Create a Pyomo model to solve the above problem with the following components:

- Variables and Constraints
    - Flow rate
    - KMnO4 required for each inlet ion
    - Alkalinity consumed for each inlet ion
    - Sludge produced for each inlet ion

- Parameters
    - Initial concentration
    - Conversion factors for KMnO4, alkalinity, and sludge from Table 20-8 and 20-10

- Expressions
    - Total KMnO4 required
    - Total alkalinity consumed
    - Total sludge produced

The model should print:
- Termination condition
- Total KMnO4 required (kg/d and lb/hr)
- Total alkalinity consumed (kg/d and lb/hr)
- Total sludge produced (kg/d and lb/hr)
- KMnO4 required for each inlet ion (kg/d)
- Alkalinity consumed for each inlet ion (kg/d)
- Sludge produced for each inlet ion (kg/d)

Components should be indexed to `["Fe", "Mn"]` where appropriate. If unit conversions are necessary, use `pyunits` (i.e., no hard-coded unit conversions).

## Required data and imports

In [2]:
# Pyomo imports
from pyomo.environ import (
    ConcreteModel,
    Var,
    Param,
    Constraint,
    Expression,
    SolverFactory,
    value,
    assert_optimal_termination,
    units as pyunits,
)

# IDAES imports
from idaes.core import FlowsheetBlock
from idaes.core.util.model_statistics import degrees_of_freedom

# WaterTAP imports
from watertap.core.solvers import get_solver

# Ion index
ions = ["Fe", "Mn"]
# Initial concentration, g/m3
conc_init = {"Fe": 5, "Mn": 2}
# Conversion to KMnO4 dose, g/g
kmno4_conv_init = {"Fe": 0.94, "Mn": 1.92}
# Conversion to alkalinity consumed, g/g
alk_conv_init = {"Fe": 1.5, "Mn": 1.21}
# Conversion to sludge produced, g/g
sludge_conv_init = {"Fe": 2.43, "Mn": 2.64}

## Create model and flowsheet

In [3]:
# create ConcreteModel and FlowsheetBlock
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

## Add parameters

In [ ]:
# add Params

m.fs.aq_conc = Param(
    ions,
    initialize=conc_init,
    units=pyunits.g / pyunits.m**3,
    doc="Aqueous concentration of ions",
)

m.fs.kmno4_conversion = Param(
    ions,
    initialize=kmno4_conv_init,
    units=pyunits.g / pyunits.g,
    doc="KMnO4 required conversion",
)

m.fs.alk_conversion = Param(
    ions,
    initialize=alk_conv_init,
    units=pyunits.g / pyunits.g,
    doc="Alkalinity consumed conversion",
)

m.fs.sludge_conversion = Param(
    ions,
    initialize=sludge_conv_init,
    units=pyunits.g / pyunits.g,
    doc="Sludge produced conversion",
)

## Add variables

In [ ]:
# add Vars

m.fs.flow_in = Var(
    initialize=1000,
    bounds=(0, 1e6),
    units=pyunits.m**3 / pyunits.d,
    doc="Daily flow rate in",
)

m.fs.kmno4_required = Var(
    ions,
    initialize=100,
    bounds=(0, 1e5),
    units=pyunits.kg / pyunits.d,
    doc="Daily KMnO4 required per ion",
)

m.fs.alk_consumed = Var(
    ions,
    initialize=100,
    bounds=(0, 1e5),
    units=pyunits.kg / pyunits.d,
    doc="Daily alkalinity consumed per ion",
)

m.fs.sludge_produced = Var(
    ions,
    initialize=100,
    bounds=(0, 1e5),
    units=pyunits.kg / pyunits.d,
    doc="Daily sludge production per ion",
)

## Create constraints

In [ ]:
# add Constraints


def rule_kmno4_req(b, i):
    return b.kmno4_required[i] == pyunits.convert(
        b.aq_conc[i] * b.flow_in * b.kmno4_conversion[i],
        to_units=pyunits.kg / pyunits.d,
    )


def rule_alk_consumed(b, i):
    return b.alk_consumed[i] == pyunits.convert(
        b.aq_conc[i] * b.flow_in * b.alk_conversion[i], to_units=pyunits.kg / pyunits.d
    )


def rule_sludge_produced(b, i):
    return b.sludge_produced[i] == pyunits.convert(
        b.aq_conc[i] * b.flow_in * b.sludge_conversion[i],
        to_units=pyunits.kg / pyunits.d,
    )


m.fs.kmno4_req_constr = Constraint(
    ions, rule=rule_kmno4_req, doc="KMnO4 requirement equation"
)
m.fs.alk_consumed_constr = Constraint(
    ions, rule=rule_alk_consumed, doc="Alkalinity consumed equation"
)
m.fs.sludge_produced_constr = Constraint(
    ions, rule=rule_sludge_produced, doc="Sludge produced equation"
)

######################################################
# Alternative way to add Constraints
######################################################

# m.fs.kmno4_req_constr = Constraint(
#     ions,
#     rule=lambda b, i: b.kmno4_required[i]
#     == pyunits.convert(
#         b.aq_conc[i] * b.flow_in * b.kmno4_conversion[i],
#         to_units=pyunits.kg / pyunits.d,
#     ),
# )

# m.fs.alk_consumed_constr = Constraint(
#     ions,
#     rule=lambda b, i: b.alk_consumed[i]
#     == pyunits.convert(
#         b.aq_conc[i] * b.flow_in * b.alk_conversion[i], to_units=pyunits.kg / pyunits.d
#     ),
# )

# m.fs.sludge_produced_constr = Constraint(
#     ions,
#     rule=lambda b, i: b.sludge_produced[i]
#     == pyunits.convert(
#         b.aq_conc[i] * b.flow_in * b.sludge_conversion[i],
#         to_units=pyunits.kg / pyunits.d,
#     ),
# )

## Create expressions

In [ ]:
# add Expressions for total KMnO4 dose, alkalinity consumed, and sludge produced

m.fs.total_kmno4_required = Expression(
    expr=sum(m.fs.kmno4_required[i] for i in ions),
    doc="Expression for total KMnO4 required",
)

m.fs.total_alk_consumed = Expression(
    expr=sum(m.fs.alk_consumed[i] for i in ions),
    doc="Expression for total alkalinity consumed",
)

m.fs.total_sludge_produced = Expression(
    expr=sum(m.fs.sludge_produced[i] for i in ions),
    doc="Expression for total sludge produced",
)

m.fs.total_kmno4_required_B = Expression(
    expr=pyunits.convert(m.fs.total_kmno4_required, to_units=pyunits.lb / pyunits.hr),
    doc="Expression for total KMnO4 required, lb/hr",
)

m.fs.total_alk_consumed_B = Expression(
    expr=pyunits.convert(m.fs.total_alk_consumed, to_units=pyunits.lb / pyunits.hr),
    doc="Expression for total alkalinity consumed, lb/hr",
)

m.fs.total_sludge_produced_B = Expression(
    expr=pyunits.convert(m.fs.total_sludge_produced, to_units=pyunits.lb / pyunits.hr),
    doc="Expression for total sludge produced, lb/hr",
)

# Similar implementations can be used for Expressions as with Constraints above

## Fix inlet flow rate

In [ ]:
# .fix inlet flow rate

m.fs.flow_in.fix(1e5)

## Check degrees of freedom

In [ ]:
# check degrees of freedom
# there should be zero

dof = degrees_of_freedom(m)
print(f"Degrees of Freedom: {dof}")

## Create `solver` 

In [ ]:
# create solver

solver = get_solver()

## Solve model and check termination condition 

In [ ]:
# solve model
# check termination condition

results = solver.solve(m)
assert_optimal_termination(results)
print(f"Model solve = {results.solver.termination_condition}\n")

## Print results 

In [ ]:
# print results
# for each ion and total

for ion in ions:
    print(f"Ion {ion}:")
    print(
        f"  KMnO4 Required: {value(m.fs.kmno4_required[ion]):.2f} {pyunits.get_units(m.fs.kmno4_required[ion])}"
    )
    print(
        f"  Alkalinity Consumed: {value(m.fs.alk_consumed[ion]):.2f} {pyunits.get_units(m.fs.alk_consumed[ion])}"
    )
    print(
        f"  Sludge Produced: {value(m.fs.sludge_produced[ion]):.2f} {pyunits.get_units(m.fs.sludge_produced[ion])}"
    )

print(
    f"\nTotal KMnO4 Required (kg/d): {value(m.fs.total_kmno4_required):.2f} {pyunits.get_units(m.fs.total_kmno4_required)}"
)
print(
    f"Total Alkalinity Consumed (kg/d): {value(m.fs.total_alk_consumed):.2f} {pyunits.get_units(m.fs.total_alk_consumed)}"
)
print(
    f"Total Sludge Produced (kg/d): {value(m.fs.total_sludge_produced):.2f} {pyunits.get_units(m.fs.total_sludge_produced)}"
)

print(
    f"\nTotal KMnO4 Required (lb/hr): {value(m.fs.total_kmno4_required_B):.2f} {pyunits.get_units(m.fs.total_kmno4_required_B)}"
)
print(
    f"Total Alkalinity Consumed (lb/hr): {value(m.fs.total_alk_consumed_B):.2f} {pyunits.get_units(m.fs.total_alk_consumed_B)}"
)
print(
    f"Total Sludge Produced (lb/hr): {value(m.fs.total_sludge_produced_B):.2f} {pyunits.get_units(m.fs.total_sludge_produced_B)}"
)